<a href="https://colab.research.google.com/github/SanjayBista1010/Classification/blob/main/TitanicBoosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load Titanic dataset from seaborn
import seaborn as sns
df = sns.load_dataset("titanic")

# Drop rows with missing target
df = df.dropna(subset=['survived'])
print(df)
# Select features and target
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
X = df[features]
y = df['survived']

# Handle missing values simply
X['age'].fillna(X['age'].median(), inplace=True)
X['embarked'].fillna('S', inplace=True)  # most common

# Identify categorical columns
cat_features = ['pclass', 'sex', 'embarked']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


     survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
886         0       2    male  27.0      0      0  13.0000        S  Second   
887         1       1  female  19.0      0      0  30.0000        S   First   
888         0       3  female   NaN      1      2  23.4500        S   Third   
889         1       1    male  26.0      0      0  30.0000        C   First   
890         0       3    male  32.0      0      0   7.7500        Q   Third   

       who  adult_male deck  embark_town alive  alo

/tmp/ipython-input-35-958390014.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['age'].fillna(X['age'].median(), inplace=True)
/tmp/ipython-input-35-958390014.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'].fillna(X['age'].median(), inplace=True)
/tmp/ipython-input-35-958390014.py:18: FutureWarning: A value is trying to be

In [36]:
!pip install catboost

In [37]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=4,
    eval_metric='Accuracy',
    random_seed=42,
    verbose=0  # Set to 100 to see logs
)

# Fit with categorical feature support
model.fit(X_train, y_train, cat_features=cat_features)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print("CatBoost Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


CatBoost Accuracy: 0.8100558659217877

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.88      0.84       105
           1       0.80      0.72      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179



In [38]:
X['sex'] = X['sex'].replace({'male': 0, 'female': 1})
X = pd.get_dummies(X, columns=['embarked'], drop_first=True,dtype=int)
X


/tmp/ipython-input-38-3544085099.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X['sex'] = X['sex'].replace({'male': 0, 'female': 1})
/tmp/ipython-input-38-3544085099.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['sex'] = X['sex'].replace({'male': 0, 'female': 1})


,pclass,sex,age,sibsp,parch,fare,embarked_Q,embarked_S
0,3,0,22.0,1,0,7.2500,0,1
1,1,1,38.0,1,0,71.2833,0,0
2,3,1,26.0,0,0,7.9250,0,1
3,1,1,35.0,1,0,53.1000,0,1
4,3,0,35.0,0,0,8.0500,0,1
...,...,...,...,...,...,...,...,...
886,2,0,27.0,0,0,13.0000,0,1
887,1,1,19.0,0,0,30.0000,0,1
888,3,1,28.0,1,2,23.4500,0,1
889,1,0,26.0,0,0,30.0000,0,0


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled, X_test_scaled

(array([[-1.61413602, -0.7243102 ,  1.25364106, ..., -0.07868358,
         -0.30335547,  0.59248936],
        [-0.40055118, -0.7243102 , -0.47728355, ..., -0.37714494,
         -0.30335547,  0.59248936],
        [ 0.81303367, -0.7243102 ,  0.21508629, ..., -0.47486697,
         -0.30335547,  0.59248936],
        ...,
        [ 0.81303367, -0.7243102 ,  0.90745614, ..., -0.35580399,
         -0.30335547,  0.59248936],
        [-1.61413602,  1.38062393, -1.1696534 , ...,  1.68320121,
         -0.30335547,  0.59248936],
        [-1.61413602, -0.7243102 , -0.63114352, ...,  0.86074761,
         -0.30335547,  0.59248936]]),
 array([[ 0.81303367, -0.7243102 , -0.09263364, ..., -0.33390078,
         -0.30335547, -1.68779402],
        [-0.40055118, -0.7243102 ,  0.13815631, ..., -0.42528387,
         -0.30335547,  0.59248936],
        [ 0.81303367, -0.7243102 , -0.7080735 , ..., -0.47486697,
         -0.30335547,  0.59248936],
        ...,
        [ 0.81303367,  1.38062393,  0.67666619, ..., -